In [ ]:
library('Seurat')
library('dplyr')
library('tidyr')
library('stringr')
library('tidyverse')
library('ggplot2')
library('pheatmap')
library('data.table')
library('ggrepel')
library('msigdbr')
library(ggvenn)
library('fgsea') #need to add to yml file
options(repr.plot.width = 5, repr.plot.height = 5) # set default plot size
options(jupyter.plot_mimetypes = 'image/png') # output SVG
options(ggrepel.max.overlaps = Inf)

## Load data

In [ ]:
SPI1a = read.csv('~/github/THP1_PrimaryMac_CRISPR/Perturbseq/DEGs_NEW/CRISPRa/SPI1.csv')
SPI1i = read.csv('~/github/THP1_PrimaryMac_CRISPR/Perturbseq/DEGs_NEW/CRISPRi/SPI1.csv')

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 8)

# Add a column to indicate the direction of regulation
SPI1a$regulation <- ifelse(SPI1a$adj_pval < 0.05 & SPI1a$lfc > 0.1, "Upregulated",
                           ifelse(SPI1a$adj_pval < 0.05 & SPI1a$lfc < -0.1, "Downregulated", "Not significant"))

specific_genes <- c("SPI1", "MT1G", "DAB1", "MT2A", "RTN1", "CYBB", "MYC", "IFI30", "SDC2", "DMXL2", "LST1")  
specific_genes_data <- SPI1a[SPI1a$name %in% specific_genes, ]

p <- ggplot(data=SPI1a, aes(x=lfc, y=-log10(pval))) +
  geom_point(aes(fill=regulation), shape = 21, stroke = 0.5, size=3) +
  scale_fill_manual(values = c("Not significant" = "darkgrey", "Upregulated" = "red", "Downregulated" = "blue")) +
  #geom_point(data=subset(SPI1a, SPI1a$name=="Cd300lb"), aes(x=lfc, y=-log10(pval))) +
  #geom_label_repel(data=SPI1a, aes(x = lfc, y = -log10(pval), label = ifelse((pval<1E-3 & abs(lfc)>1), name,"")), box.padding=0.5, size=6) +
  geom_label_repel(data = specific_genes_data, aes(x = lfc, y = -log10(pval), label = name), 
                   box.padding = 0.5, size = 5, color = "black", fill = "white") +
  theme_classic() +
  #geom_hline(yintercept = -log10(0.05), linetype=2, color="black") +
  geom_hline(yintercept = 0) +
  geom_vline(xintercept = 0) +
  #geom_vline(xintercept = c(-2, 2), linetype=2, color="black") +
  labs(x = "log2 Fold Change", y = "-log10(p-value)") +
  theme(legend.position="none",
       axis.title.x = element_text(size = 16),
       axis.text.x = element_text(size = 16),
       axis.title.y = element_text(size = 16),
       axis.text.y = element_text(size = 16)) + xlim(-5,5) #+ ggtitle("SPI1 CRISPRa vs. NT")
ggsave("SPI1_CRISPRa_volcanoplot.pdf", plot = p, width = 10, height = 8)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 8)

# Add a column to indicate the direction of regulation
SPI1i$regulation <- ifelse(SPI1i$adj_pval < 0.05 & SPI1i$lfc > 0.1, "Upregulated",
                           ifelse(SPI1i$adj_pval < 0.05 & SPI1i$lfc < -0.1, "Downregulated", "Not significant"))

specific_genes <- c("SPI1", "APOE", "CHIT1", "APOC1", "LILRB4", "CSF1R", "ELMO1", "IFI30", "MSR1", "TGFB1", "IFIT1", "MSI2", "TNFAIP6", "PTGDS")  
specific_genes_data <- SPI1i[SPI1i$name %in% specific_genes, ]

ggplot(data=SPI1i, aes(x=lfc, y=-log10(pval))) +
  geom_point(aes(fill=regulation), shape = 21, stroke = 0.5, size=3) +
  scale_fill_manual(values = c("Not significant" = "darkgrey", "Upregulated" = "red", "Downregulated" = "blue")) +
  #geom_point(data=subset(SPI1a, SPI1a$name=="Cd300lb"), aes(x=lfc, y=-log10(pval))) +
  #geom_label_repel(data=SPI1a, aes(x = lfc, y = -log10(pval), label = ifelse((pval<1E-3 & abs(lfc)>1), name,"")), box.padding=0.5, size=6) +
  geom_label_repel(data = specific_genes_data, aes(x = lfc, y = -log10(pval), label = name), 
                   box.padding = 0.5, size = 5, color = "black", fill = "white") +
  theme_classic() +
  #geom_hline(yintercept = -log10(0.05), linetype=2, color="black") +
  geom_hline(yintercept = 0) +
  geom_vline(xintercept = 0) +
  #geom_vline(xintercept = c(-2, 2), linetype=2, color="black") +
  labs(x = "log2 Fold Change", y = "-log10(p-value)") +
  theme(legend.position="none",
       axis.title.x = element_text(size = 16),
       axis.text.x = element_text(size = 16),
       axis.title.y = element_text(size = 16),
       axis.text.y = element_text(size = 16)) + xlim(-5,5) #+ ggtitle("SPI1 CRISPRa vs. NT")
#ggsave("SPI1_CRISPRi_volcanoplot.pdf", plot = p, width = 10, height = 8)

## find common elements

In [ ]:
SPI1i_DEGs = SPI1i[SPI1i$adj_pval < 0.05,'name']
SPI1a_DEGs = SPI1a[SPI1a$adj_pval < 0.05,'name']

In [ ]:
common_elements <- intersect(SPI1i_DEGs, SPI1a_DEGs)

In [ ]:
data <- list(
  CRISPRi = SPI1i_DEGs,
  CRISPRa = SPI1a_DEGs
)

# Create the Venn diagram
ggvenn(data, fill_color = c('skyblue', 'salmon'),stroke_size = 2, set_name_size = 8, show_percentage = FALSE)
#ggsave("SPI1_venndiagram.pdf", plot = p, width = 6, height = 6)

## 4 way plot

In [ ]:
Zp<-function(p){-qnorm (p/2)} # Transformation to Z scores

In [ ]:
SPI1i$Zscore <- Zp(SPI1i$adj_pval)*sign(SPI1i$lfc)
SPI1a$Zscore <- Zp(SPI1a$adj_pval)*sign(SPI1a$lfc)

In [ ]:
dataSele2 = merge(SPI1i[c('name', 'Zscore')], SPI1a[c('name', 'Zscore')], by = 'name')

dataSeleUp = subset(dataSele2, dataSele2[,2]>2 & dataSele2[,3]< -2)
dataSeleDown = subset(dataSele2, dataSele2[,2]< -2 & dataSele2[,3]> 2)
ggplot() +
  geom_point(data=dataSele2, aes(x = dataSele2[,2], y = dataSele2[,3]), shape=1, size=2) +
  geom_point(data=dataSeleUp, aes(x = dataSeleUp[,2], y = dataSeleUp[,3]), colour="red") +
  geom_point(data=dataSeleDown, aes(x = dataSeleDown[,2], y = dataSeleDown[,3]), colour="blue") +
  geom_hline(yintercept=0, color = "black") +
  geom_vline(xintercept=0, color = "black") +
  geom_hline(yintercept=c(-2,2), linetype="dashed", color = "black") +
  geom_vline(xintercept=c(-2,2), linetype="dashed", color = "black") +
  geom_label_repel(data=dataSele2, aes(x = dataSele2[,2], y = dataSele2[,3], label=ifelse(((dataSele2[,2]>2 & dataSele2[,3]< -2) | (dataSele2[,2]< -2 & dataSele2[,3]> 2)),name,"")), box.padding=0.2, size=4) +
  theme_minimal() + xlim(-12,12) + ylim(-5,8) +
  xlab('SPI1 inhibition') +
  ylab('SPI1 activation') + 
  theme(
    axis.title.x = element_text(size = 14),
    axis.title.y = element_text(size = 14),
    axis.text.x = element_text(size = 12),
    axis.text.y = element_text(size = 12)
    #plot.title = element_text(size = 16, face = "bold"),
    #plot.margin = margin(1, 1, 1, 1, "cm")
  )

#ggsave("SPI1_4way_plot.pdf", plot = p, width = 10, height = 8)

## GSEA

In [ ]:
library(clusterProfiler)
library(org.Hs.eg.db)
library(msigdbr)

In [ ]:
gene_symbols <- SPI1i$name
entrez_ids <- bitr(gene_symbols, fromType = "SYMBOL", toType = "ENTREZID", OrgDb = org.Hs.eg.db)

In [ ]:
SPI1i = merge(SPI1i, entrez_ids, by.x = "name", by.y = "SYMBOL", all.x = TRUE)

In [ ]:
gene_symbols <- SPI1a$name
entrez_ids <- bitr(gene_symbols, fromType = "SYMBOL", toType = "ENTREZID", OrgDb = org.Hs.eg.db)

In [ ]:
SPI1a = merge(SPI1a, entrez_ids, by.x = "name", by.y = "SYMBOL", all.x = TRUE)

In [ ]:
msig_h <- msigdbr(species = "Homo sapiens", category = "H") %>%
    dplyr::select(gs_name, human_gene_symbol) %>%
    dplyr::rename(ont = gs_name, gene = human_gene_symbol)
#msig_h

In [ ]:
SPI1i_ordered = SPI1i[order(SPI1i$Zscore),]

In [ ]:
ordered_genes <- SPI1i_ordered$Zscore
names(ordered_genes) <- SPI1i_ordered$name
ordered_genes <- sort(ordered_genes, decreasing = TRUE)

In [ ]:
msig_gsea <- GSEA(ordered_genes, TERM2GENE = msig_h) %>%
    as_tibble

In [ ]:
top_pathways <- msig_gsea %>%
  arrange(p.adjust) %>%
  head(30)

In [ ]:
top_pathways$pathway <- factor(top_pathways$Description, levels = top_pathways$Description[order(top_pathways$NES)])
top_pathways$pathway = sub("HALLMARK_", "", top_pathways$pathway)
top_pathways$pathway <- factor(top_pathways$pathway, levels = top_pathways$pathway[order(top_pathways$NES)])

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 5)

ggplot(top_pathways, aes(x = NES, y = pathway, fill = p.adjust)) +
  geom_bar(stat = "identity") +
  scale_fill_gradient(low = "indianred2", high = "dodgerblue2", , limits = c(0, 0.1)) +
  labs(x = "Normalized Enrichment Score (NES)", y = "") +
  theme_classic() +
  theme(
    axis.title.x = element_text(size = 16),
    axis.text.x = element_text(size = 16),
    axis.title.y = element_text(size = 16),
    axis.text.y = element_text(size = 16)
  ) +
  scale_size_continuous(range = c(3, 10))

ggsave("SPI1_CRISPRi_barplot.pdf", width = 12, height = 5)

In [ ]:
SPI1a_ordered = SPI1a[order(SPI1a$Zscore),]

In [ ]:
ordered_genes <- SPI1a_ordered$Zscore
names(ordered_genes) <- SPI1a_ordered$name
ordered_genes <- sort(ordered_genes, decreasing = TRUE)

In [ ]:
msig_gsea <- GSEA(ordered_genes, TERM2GENE = msig_h) %>%
    as_tibble

In [ ]:
top_pathways <- msig_gsea %>%
  arrange(p.adjust) %>%
  head(15)

In [ ]:
top_pathways$pathway <- factor(top_pathways$Description, levels = top_pathways$Description[order(top_pathways$NES)])
top_pathways$pathway = sub("HALLMARK_", "", top_pathways$pathway)
top_pathways$pathway <- factor(top_pathways$pathway, levels = top_pathways$pathway[order(top_pathways$NES)])

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 5)

ggplot(top_pathways, aes(x = NES, y = pathway, fill = p.adjust)) +
  geom_bar(stat = "identity") +
  scale_fill_gradient(low = "indianred2", high = "dodgerblue2", , limits = c(0, 0.1)) +
  labs(x = "Normalized Enrichment Score (NES)", y = "") +
  theme_classic() +
  theme(
    axis.title.x = element_text(size = 16),
    axis.text.x = element_text(size = 16),
    axis.title.y = element_text(size = 16),
    axis.text.y = element_text(size = 16)
  ) +
  scale_size_continuous(range = c(3, 10))

ggsave("SPI1_CRISPRa_barplot.pdf", width = 12, height = 5)